In [18]:
import sys
import os

SCRIPT_DIR = os.path.dirname(os.path.abspath('getters/get_data.py'))
sys.path.append(os.path.dirname(SCRIPT_DIR))

In [26]:
import pandas as pd
from getters.get_data import GetExpectedInflation
import getters.urls_dict as u

### Carregando dados da inflação esperada através de scraping

In [39]:
    url = u.urls.get('Inflação Esperada')
    headers = u.headers
    get_exp = GetExpectedInflation(url, headers)

    headings, ano, mes, value = get_exp.run()

Scraping url: http://www.ipeadata.gov.br/ExibeSerie.aspx?serid=1693254712&module=M
Approved access
['Expectativa média de Inflação - IPCA - taxa acumulada para os próximos doze meses']
6,40
6,57
6,53
6,49
5,57
4,84
4,76
4,70
4,94
4,90
4,37
4,57
5,08
5,40
6,28
8,94
12,47
13,24
11,72
11,07
9,90
9,18
8,23
7,08
6,39
6,32
6,56
6,15
5,80
5,92
6,01
5,74
5,54
5,60
5,98
6,33
6,36
6,27
6,24
6,22
6,30
6,08
5,74
5,61
5,67
5,87
5,46
4,85
4,86
4,70
4,74
4,73
4,61
4,46
4,62
4,40
4,31
4,06
4,10
3,98
4,28
4,40
3,93
4,00
4,14
4,08
4,03
3,81
3,74
3,47
3,40
3,47
3,55
3,83
3,75
3,71
3,85
4,45
4,44
4,28
4,36
4,56
5,08
5,60
5,49
5,16
5,01
5,40
5,45
4,93
4,70
4,57
4,14
4,21
4,14
4,06
4,09
3,97
4,19
4,32
4,35
4,40
4,68
4,66
4,70
4,86
4,88
4,74
4,82
4,98
5,31
5,42
5,61
5,52
5,67
5,61
5,61
4,86
4,88
4,74
5,35
5,47
5,80
5,56
5,53
5,35
5,30
5,26
5,43
5,57
5,52
5,38
5,58
5,68
5,64
5,47
5,36
5,63
5,70
5,50
5,53
5,56
5,62
5,74
5,66
6,06
6,16
6,25
6,14
6,11
6,02
6,17
6,42
6,22
6,03
6,01
5,95
6,15
6,33
6,41
6,56
6,73
6

In [53]:
exp = pd.DataFrame(data={'ano':ano,'mes':mes,'valor':value})
exp.rename(columns={'valor':headings[0].split('-')[0]}, inplace=True)

In [54]:
exp.head()

,ano,mes,Expectativa média de Inflação
0,2001,07,"6,40"
1,2001,08,"6,57"
2,2001,09,"6,53"
3,2001,10,"6,49"
4,2001,11,"5,57"


### Carregando dados do IPCA obtidos através de scraping

In [56]:
FILE = 'IPCA-serie-historica.csv/mes_brasil.csv'

In [57]:
ipca = pd.read_csv('../getters/Data/extracted/'+FILE)

In [58]:
ipca.head()

,ano,mes,indice,variacao_mensal,variacao_trimestral,variacao_semestral,variacao_anual,variacao_doze_meses
0,1979,12,7.618300e-09,NaN,NaN,NaN,NaN,NaN
1,1980,1,8.122300e-09,6.62,NaN,NaN,6.62,NaN
2,1980,2,8.497300e-09,4.62,NaN,NaN,11.54,NaN
3,1980,3,9.010400e-09,6.04,18.27,NaN,18.27,NaN
4,1980,4,9.486700e-09,5.29,16.80,NaN,24.53,NaN


### Carregando dados da taxa de juros básica via scraping